In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import efficientnet,EfficientNetB0

(x_train,y_train),(x_val,y_val)=fashion_mnist.load_data()
x_train,y_train,x_val,y_val=x_train[:20000],y_train[:20000],x_train[57000:],y_train[57000:]


data_augmentation=tf.keras.Sequential([
    tf.keras.layers.Resizing(32,32),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomFlip("horizontal")
])

x_train,x_val=x_train/255.0,x_val/255.0
x_train,x_val=tf.image.grayscale_to_rgb(tf.expand_dims(x_train,-1)),tf.image.grayscale_to_rgb(tf.expand_dims(x_val,-1))

train=(
    tf.data.Dataset.from_tensor_slices((x_train,y_train))
    .shuffle(10000)
    .map(lambda x,y:(data_augmentation(x),y))
    .map(lambda x,y:(efficientnet.preprocess_input(x),y))
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
    )
val=(
    tf.data.Dataset.from_tensor_slices((x_val,y_val))
    .map(lambda x,y:(tf.image.resize(x,[32,32]),y))
    .map(lambda x,y:(efficientnet.preprocess_input(x),y))
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
    )

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [2]:
model=EfficientNetB0(weights=None,include_top=False)
out=tf.keras.layers.Conv2D(10,kernel_size=1,activation="softmax")(model.output)
gb=tf.keras.layers.GlobalAveragePooling2D()(out)
model=tf.keras.Model(inputs=model.input,outputs=gb)

model.compile(optimizer="nadam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

history=model.fit(train,validation_data=val,epochs=5)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 129s 52ms/step - accuracy: 0.3899 - loss: 1.7335 - val_accuracy: 0.2260 - val_loss: 2.6944
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.6866 - loss: 0.8668 - val_accuracy: 0.7640 - val_loss: 0.6233
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.7371 - loss: 0.7337 - val_accuracy: 0.7570 - val_loss: 0.6752
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.7338 - loss: 0.7874 - val_accuracy: 0.6600 - val_loss: 0.9163
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.7073 - loss: 0.8452 - val_accuracy: 0.7620 - val_loss: 0.6372
